# Basic setup

In [ ]:
!pip install --quiet -U --force transformers ipywidgets langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph 

## Setup check-up

In [ ]:
try:
    import langchain
    print("Langchain est installé, version:", langchain.__version__)
except ImportError:
    print("Langchain n'est pas installé.")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

try:
    from transformers import pipeline
    print("Hugging Face Transformers est installé.")
except ImportError:
    print("Hugging Face Transformers n'est pas installé.")

In [ ]:
try:
    import tensorflow as tf
    print("TensorFlow est installé, version:", tf.__version__)
except ImportError:
    print("TensorFlow n'est pas installé.")

In [ ]:
import torch

torch.cuda.is_available()

# Setup the LLM manager

In [ ]:
!docker create --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name llm_server ollama/ollama

In [ ]:
!docker ps

## Run or stop the LLM server

In [ ]:
!docker start llm_server

In [ ]:
!docker stop llm_server

In [ ]:
!yes | docker system prune -a

## Pull the desired modules

In [ ]:
!docker exec -it llm_server ollama pull llama3.2

In [ ]:
!docker exec -it llm_server ollama pull qwen2.5:32b-instruct-fp16

In [ ]:
!docker exec -it llm_server ollama pull mistral-small

In [ ]:
!docker exec -it llm_server ollama pull nomic-embed-text

## Test the pulled model

In [60]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2")

In [61]:
llm.invoke("The first publicly large scale available chat AI was ...")

'The first publicly large-scale available chat AI was ELIZA, developed in 1966 by Joseph Weizenbaum at the Massachusetts Institute of Technology (MIT).'

# Un simple chaîne

In [95]:
from langchain_ollama import OllamaLLM

sc_llm = OllamaLLM(model="llama3.2")

In [96]:
# Importing necessary modules from the LangChain core library
from langchain_core.messages import HumanMessage, AIMessage  # To manage message types (human/system)
from langchain_core.output_parsers import StrOutputParser  # To parse the output from the model
from langchain_core.prompts import ChatPromptTemplate  # To create prompt templates for the LLM

# Defining a system message template with a prompt that explains a specific topic
system_template = "Explain to the user the story of the {topic} he asked for."

# Creating a ChatPromptTemplate object using system and user messages
# The system message uses the 'system_template', while the user message contains the user's input as 'text'
sc_prompt_template = ChatPromptTemplate.from_messages(
    [system_template, "{text}"]
)

# Creating a string output parser to handle the output as plain text
sc_parser = StrOutputParser()

# Defining a chain that connects the prompt template, the LLM, and the output parser
# This chain processes the prompt, generates the response, and parses it
simplechain = sc_prompt_template | sc_llm | sc_parser

In [97]:
simplechain.invoke({"topic": "Natural Language Processing", "text": "Tell me more about IBM Watson"})

'IBM Watson is a cloud-based artificial intelligence (AI) platform that uses natural language processing (NLP) and machine learning algorithms to analyze and understand human language. Developed by IBM in 2007, Watson was originally designed to compete on the game show "Jeopardy!" against two human champions.\n\nHere\'s how it worked: Watson was trained on a massive database of text from various sources, including books, articles, and other information. The platform used NLP techniques to analyze and understand the language in the questions asked by the contestants. It could identify entities such as names, dates, and locations, extract key phrases, and even recognize nuances like idioms and metaphors.\n\nOn "Jeopardy!", Watson was presented with a series of questions in the form of clues, which it would then answer correctly in the format of "What is [answer]?" The human champions were surprised by Watson\'s accuracy and speed, and it ultimately defeated them to win the show.\n\nAfter

# Exemples Langchain

## Structured Output Parser

In [9]:
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama

# Définition d'un modèle Pydantic pour la sortie
class ResponseFormatter(BaseModel):
    answer: str = Field(description="La réponse à la question de l'utilisateur.")
    followup_question: str = Field(description="Une question de suivi que l'utilisateur pourrait poser.")

# Initialiser le modèle de chat
# llm = ChatOllama(model="mistral-small", format="json")
so_llm = ChatOllama(model="llama3.2")

# Lier le modèle à l'output structuré
structured_model = so_llm.with_structured_output(ResponseFormatter,include_raw=True)

# Exemple d'invocation
response = structured_model.invoke("Quel est le capital de la France ?")

# Afficher la sortie structurée
print(response)

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-06T18:31:40.443422938Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'ResponseFormatter', 'arguments': {'answer': 'Paris', 'followup_question': ''}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 172617532, 'load_duration': 15431599, 'prompt_eval_count': 190, 'prompt_eval_duration': 7285000, 'eval_count': 24, 'eval_duration': 106020000}, id='run-b183d3ba-b3df-42f1-961f-2bd67ecb89cf-0', tool_calls=[{'name': 'ResponseFormatter', 'args': {'answer': 'Paris', 'followup_question': ''}, 'id': 'c406a4cc-dbc9-4262-bcf7-fdf51281cbd5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 24, 'total_tokens': 214}), 'parsed': ResponseFormatter(answer='Paris', followup_question=''), 'parsing_error': None}


In [7]:
print(response.followup_question)

Quelle est la préfecture administrative de la région Île-de-France ?


In [8]:
parser = PydanticOutputParser(pydantic_object=ResponseFormatter)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"answer": {"description": "La réponse à la question de l\'utilisateur.", "title": "Answer", "type": "string"}, "followup_question": {"description": "Une question de suivi que l\'utilisateur pourrait poser.", "title": "Followup Question", "type": "string"}}, "required": ["answer", "followup_question"]}\n```'

## Le Templates

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Donne-moi une blague sur {sujet}.")
response = prompt_template.invoke({"sujet": "les chats"})
response

StringPromptValue(text='Donne-moi une blague sur les chats.')

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Tu es un assistant utile."),
    ("user", "Raconte-moi une blague sur les chats.")
])
response = prompt_template.invoke({})
response

ChatPromptValue(messages=[SystemMessage(content='Tu es un assistant utile.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Raconte-moi une blague sur les chats.', additional_kwargs={}, response_metadata={})])

## Les embeddings

In [26]:
from langchain.embeddings import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Initialize Ollama embeddings model
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Sample text for vector store
text = "LangChain is the framework for building context-aware reasoning applications"
vector = embeddings.embed_query(text)

print(len(vector))
vector[10:20] # j'affiche que 10 dimensions sur 768, pour la démonstration..

768


[-0.33447709679603577,
 0.6420562267303467,
 1.4902821779251099,
 0.5697729587554932,
 0.009297564625740051,
 0.430616557598114,
 -0.6175608038902283,
 -0.03231702744960785,
 -0.4382496178150177,
 0.08225058764219284]

## Les Vector Stores

In [52]:
# Create an in-memory vector store and populate it with embeddings of the sample text
vectorstore = InMemoryVectorStore.from_texts(
    [
        "It's cool to be a BBS student",
        "I wish I could have chicken wings right now",
        "Sorry, wasn't listening"
    ],
    embedding=embeddings
)

In [53]:
for doc in vectorstore.similarity_search(query="fried chicken",k=1):
    print(f"* {doc.page_content} [{doc.metadata}]")

* I wish I could have chicken wings right now [{}]


In [54]:
for doc, score in vectorstore.similarity_search_with_score(query="fried chicken",k=2):
    print(f"* [SIM={(score*100):3f}%] {doc.page_content} [{doc.metadata}]")

* [SIM=54.748526%] I wish I could have chicken wings right now [{}]
* [SIM=38.325593%] It's cool to be a BBS student [{}]


## Les retrievers

In [ ]:
!pip install wikipedia

In [ ]:
from langchain_community.retrievers import WikipediaRetriever

wikiretriever = WikipediaRetriever()
documents = wikiretriever.invoke("intelligence artificielle")
documents

### Vector Store Retrievers

In [62]:
# Use the vector store as a retriever (Can be “similarity” (default), “mmr”, or “similarity_score_threshold”)
mmrretriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
)

# Query the retriever with a relevant question
retrieved_documents = mmrretriever.invoke("Fried chicken?")

# Show the retrieved document's content
retrieved_documents

[Document(id='d289ad5e-017b-44d7-af4b-87208afd7083', metadata={}, page_content='I wish I could have chicken wings right now')]

In [63]:
# Create an in-memory vector store and populate it with embeddings of the sample text
wikivectorstore = InMemoryVectorStore.from_documents(
    documents,
    embedding=embeddings
)

# Use the vector store as a retriever (all defaults to cosine similarity)
wikiretriever = wikivectorstore.as_retriever()

# Query the retriever with a relevant question
retrieved_documents = wikiretriever.invoke("Any knowledge about teh Dartmouth conference?")

# Show the retrieved document's content
retrieved_documents

[Document(id='b1454c20-99dd-4191-b98b-80cd3296e0d4', metadata={'title': 'Yann LeCun', 'summary': 'Yann André LeCun ( lə-KUN, French: [ləkœ̃]; originally spelled Le Cun; born 8 July 1960) is a French-American computer scientist working primarily in the fields of machine learning, computer vision, mobile robotics and computational neuroscience. He is the Silver Professor of the Courant Institute of Mathematical Sciences at New York University and Vice-President, Chief AI Scientist at Meta.\nHe is well known for his work on optical character recognition and computer vision using convolutional neural networks (CNNs). He is also one of the main creators of the DjVu image compression technology (together with Léon Bottou and Patrick Haffner). He co-developed the Lush programming language with Léon Bottou.\nIn 2018, LeCun, Yoshua Bengio, and Geoffrey Hinton, received the Turing Award for their work on deep learning. The three are sometimes referred to as the "Godfathers of AI" and "Godfathers

## Tools

In [67]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2", temperature=0)
query = "What is 3 * 12? Also, what is 11 + 49?"

In [74]:
# Tools déclarés 
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field

class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

available_tools = [add, multiply]
llm_with_tools = llm.bind_tools(available_tools)
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-06T21:22:53.229805053Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': '3', 'b': '12'}}}, {'function': {'name': 'add', 'arguments': {'a': '11', 'b': '49'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 262830504, 'load_duration': 15371968, 'prompt_eval_count': 236, 'prompt_eval_duration': 7194000, 'eval_count': 44, 'eval_duration': 196538000}, id='run-35eb4f94-a887-41c5-89a4-d09530cbb44a-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '12'}, 'id': 'bf260269-63e3-4927-ba90-5c426a508a34', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': '11', 'b': '49'}, 'id': 'a4c53aed-bd86-4f26-8c50-aae502a2d5db', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 44, 'total_tokens': 280})

### Tool output parser

In [75]:
chain = llm_with_tools | PydanticToolsParser(tools=available_tools)

response = chain.invoke(query)
response

[multiply(a=3, b=12), add(a=11, b=49)]

## Tracing

In [89]:
from typing import Any, Dict, List, Optional, Sequence
from uuid import UUID

# from langchain_ollama import OllamaLLM
# from langchain_core.callbacks.base import BaseCallbackHandler
# from langchain_core.agents import AgentAction, AgentFinish
# from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.schema import GenerationChunk, ChatGenerationChunk, RetryCallState
# from langchain_core.retrievers import Document

from langchain_ollama import OllamaLLM
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages.base import BaseMessage
from langchain_core.outputs.llm_result import LLMResult
from langchain_core.outputs.chat_generation import ChatGenerationChunk
from langchain_core.outputs.generation import GenerationChunk
from langchain_core.documents.base import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_ollama import OllamaLLM

In [111]:
class LoggingHandler(BaseCallbackHandler):
    ##### GENERAL CASE INSPECTIONS
    # def on_retry(
    #     self,
    #     retry_state: RetryCallState,
    #     *,
    #     run_id: UUID,
    #     parent_run_id: Optional[UUID] = None,
    #     **kwargs: Any
    # ) -> Any:
    #     print(f"[Run ID: {run_id}] Retry occurred. Retry state: {retry_state}")

    def on_text(
        self,
        text: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Text event: {text}")

    ##### AGENTS INSPECTIONS 
    def on_agent_action(
        self,
        action: AgentAction,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Agent Action: {action}")

    def on_agent_finish(
        self,
        finish: AgentFinish,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Agent Finished: {finish}")

    ##### CHAINS INSPECTIONS
    def on_chain_end(
        self,
        outputs: Dict[str, Any],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Chain Ended with outputs: {outputs}")

    def on_chain_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Chain Error: {error}")

    def on_chain_start(
        self,
        serialized: Optional[Dict[str, Any]],  # Allow serialized to be Optional
        inputs: Dict[str, Any],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any
    ) -> Any:
        if serialized is None:
            chain_name = "Unnamed Chain"
            print(f"[Run ID: {run_id}] WARNING: Received None for 'serialized' in on_chain_start.")
        else:
            chain_name = serialized.get("name", "Unnamed Chain")

        print(f"[Run ID: {run_id}] INFO: Chain '{chain_name}' started with inputs: {inputs}")


    ##### CHAT MODEL INSPECTIONS
    def on_chat_model_start(
        self,
        serialized: Dict[str, Any],
        messages: List[List[BaseMessage]],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any
    ) -> Any:
        model_name = serialized.get("model", "Unnamed Model")
        print(f"[Run ID: {run_id}] Chat Model '{model_name}' started with messages: {messages}")

    ##### CUSTOM EVENTS INSPECTIONS
    def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Custom Event '{name}' triggered with data: {data}")

    ##### LLM INSPECTIONS
    def on_llm_end(
        self,
        response: Any,  # Adjusted to Any since OllamaLLM might return different types
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] LLM Ended with response: {response}")

    def on_llm_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] LLM Error: {error}")

    def on_llm_new_token(
        self,
        token: str,
        *,
        chunk: Optional[GenerationChunk] = None,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] New LLM Token: {token}")

    def on_llm_start(
        self,
        serialized: Dict[str, Any],
        prompts: List[str],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any
    ) -> Any:
        llm_name = serialized.get("model", "Unnamed LLM")
        print(f"[Run ID: {run_id}] LLM '{llm_name}' started with prompts: {prompts}")

    ##### RETRIEVERS INSPECTIONS
    def on_retriever_end(
        self,
        documents: Sequence[Document],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Retriever Ended with documents: {documents}")

    def on_retriever_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Retriever Error: {error}")

    def on_retriever_start(
        self,
        serialized: Dict[str, Any],
        query: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any
    ) -> Any:
        retriever_name = serialized.get("name", "Unnamed Retriever")
        print(f"[Run ID: {run_id}] Retriever '{retriever_name}' started with query: '{query}'")

    ##### TOOLS INSPECTIONS
    def on_tool_end(
        self,
        output: Any,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Tool Ended with output: {output}")

    def on_tool_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any
    ) -> Any:
        print(f"[Run ID: {run_id}] Tool Error: {error}")

    def on_tool_start(
        self,
        serialized: Dict[str, Any],
        input_str: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        inputs: Optional[Dict[str, Any]] = None,
        **kwargs: Any
    ) -> Any:
        tool_name = serialized.get("name", "Unnamed Tool")
        print(f"[Run ID: {run_id}] Tool '{tool_name}' started with input: '{input_str}'")

In [112]:
# Initialize the callback handler
callbacks = [LoggingHandler()]

# Invoke the chain with input and callback configuration
simplechain.invoke(
    {"topic": "Natural Language Processing", "text": "Tell me more about IBM Watson"},
    config={"callbacks": callbacks}
)

[Run ID: b54cc9f2-6d1a-4d93-b5ef-af7695e8d630] WARNING: Received None for 'serialized' in on_chain_start.
[Run ID: b54cc9f2-6d1a-4d93-b5ef-af7695e8d630] INFO: Chain 'Unnamed Chain' started with inputs: {'topic': 'Natural Language Processing', 'text': 'Tell me more about IBM Watson'}
[Run ID: e1ca1b72-cea3-408c-8aa6-c565c75194fc] INFO: Chain 'ChatPromptTemplate' started with inputs: {'topic': 'Natural Language Processing', 'text': 'Tell me more about IBM Watson'}
[Run ID: e1ca1b72-cea3-408c-8aa6-c565c75194fc] Chain Ended with outputs: messages=[HumanMessage(content='Explain to the user the story of the Natural Language Processing he asked for.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me more about IBM Watson', additional_kwargs={}, response_metadata={})]
[Run ID: 6b7ba8c0-8124-4528-aa1f-bdc1606fd0d6] LLM 'Unnamed LLM' started with prompts: ['Human: Explain to the user the story of the Natural Language Processing he asked for.\nHuman: Tell me more about I

"The story of NLP (Natural Language Processing) is a fascinating one that has evolved over decades.\n\n**What is NLP?**\n\nNLP is a subfield of artificial intelligence (AI) that deals with the interaction between computers and humans in natural language. It's a branch of AI research that aims to enable machines to understand, interpret, and generate human-like language.\n\n**Early Beginnings:**\n\nThe concept of NLP dates back to the 1950s, when Alan Turing proposed the idea of machine translation. However, it wasn't until the 1960s that NLP began to take shape as a distinct field.\n\n**Key Milestones:**\n\n1. **Chomsky's Transformational Grammar (1957):** Noam Chomsky introduced the concept of transformational grammar, which laid the foundation for modern NLP.\n2. **WordNet (1993):** A large lexical database that enabled computers to understand word relationships and semantics.\n3. **Deep Learning (2000s):** The resurgence of deep learning techniques in the 2010s revolutionized NLP, e

# Chat interface

In [ ]:
# Importing the ChatOllama class from the langchain_ollama module
from langchain_ollama import ChatOllama

# Initializing a language model (LLM) using the ChatOllama class
# max_tokens limits the response to 1024 tokens
# temperature is set to 0, meaning deterministic responses with no randomness
llm = ChatOllama(model="llama3.2", max_tokens=1024, temperature=0)


In [ ]:
# Importing Annotated from typing to allow advanced type annotations
from typing import Annotated
# Importing TypedDict from typing_extensions to define a dictionary with fixed keys and value types
from typing_extensions import TypedDict

# Importing AnyMessage from langchain_core.messages to define a general message type
from langchain_core.messages import AnyMessage

# Importing necessary components from langgraph to build a state graph
from langgraph.graph import StateGraph, START, END  # START and END mark the flow of states in the graph
from langgraph.graph.message import add_messages  # add_messages defines how messages should be added to the state

# Defining the State class as a TypedDict
# This class represents the structure of the state, with one key: "messages"
class State(TypedDict):
    # The "messages" key stores a list of AnyMessage instances
    # The Annotated type allows the `add_messages` function to handle how messages are updated
    # In this case, `add_messages` appends new messages to the existing list
    messages: Annotated[list[AnyMessage], add_messages]

# Creating an instance of StateGraph using the defined State structure
# This graph will manage the state transitions, where each state holds the messages
graph_builder = StateGraph(State)


In [ ]:
# Defining the chatbot function which takes the current state as input
# The state is expected to be of type "State", containing a list of messages
def chatbot(state: State):
    # The function returns a dictionary with the key "messages"
    # It invokes the language model (llm) on the list of messages in the current state
    # The llm.invoke() method generates a response based on the input messages
    return {"messages": [llm.invoke(state["messages"])]}

# Adding an edge to the graph from the START node to the "chatbot" node
# This signifies that the process starts at the START node and moves to the "chatbot" node
graph_builder.add_edge(START, "chatbot")

# Adding the "chatbot" node to the graph, which points to the chatbot function
# Whenever this node is used, the chatbot function will be called
graph_builder.add_node("chatbot", chatbot)

# Adding an edge from the "chatbot" node to the END node
# This means that once the "chatbot" function is executed, the process moves to the END node, signaling completion
graph_builder.add_edge("chatbot", END)

# Compiling the graph to make it ready for execution
# This graph consists of nodes and edges, representing the flow of state transitions and function calls
graph = graph_builder.compile()


In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Importing AIMessage and HumanMessage to represent different types of messages
from langchain_core.messages import AIMessage, HumanMessage

# Defining a function to send user input to the chatbot and process responses
def send_msg(user_input: str):
    # Creating a dictionary with a "messages" key containing a list of HumanMessage objects
    # The content of the HumanMessage is set to the user's input
    messages = {"messages": [HumanMessage(content=user_input)]}
    
    # Streaming events from the graph by providing the user messages as input
    for event in graph.stream(input=messages):
        # Looping through the event's values (responses generated by the assistant)
        for value in event.values():
            # Printing the last message in the list of responses from the assistant
            print("Assistant:", value["messages"][-1].content)

# A continuous loop to interact with the chatbot
while True:
    try:
        # Asking for user input via the console
        user_input = input("User: ")
        # Checking if the user wants to quit by typing "quit", "exit", or "q"
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break  # Exiting the loop if the user wants to quit

        # Sending the user input to the send_msg function to get a response from the chatbot
        send_msg(user_input)

    except:
        # Fallback behavior if input() is not available (e.g., in some environments or for testing)
        # In this case, the user input is hardcoded to a specific question
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        # Sending the hardcoded input to the chatbot
        send_msg(user_input)
        break  # Ending the loop after this fallback interaction
